In [2]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

### Plotting data question dari bulan februari sampai april 2020

In [3]:
df = pd.read_csv('question_philoit.csv')
df = df[['created_at','content']]
df['created_at']= pd.to_datetime(df['created_at'])
df['content'] = df['content'].apply(str)
df = df[(df['created_at'] > '2020-02-01') & (df['created_at'] < '2020-04-30')]
df.head
df['content']

1788      Apa yang seharusnya kita kerjakan saat liburan?
1789                     Apa makanan khas daerah kalian ?
1790                     Apa makanan khas daerah kalian ?
1791                     Apa makanan khas daerah kalian ?
1792    Menurut kamu yang akan juara club mana ditahun...
                              ...                        
4921            Apa dampak covid 19 pada kehidupan kalian
4922    Apa saja kesulitan kalian bekerja dan belajar ...
4923    Apa saja kemudahan kalian bekerja dan belajar ...
4924    Bagaimana cara kalian agar tidak bosan selama ...
4925    Apa olahraga yang mudah dilakukan selama di ru...
Name: content, Length: 3138, dtype: object

In [4]:
# Remove punctuation
df['df_text_processed'] = \
df['content'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
df['df_text_processed'] = \
df['df_text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
df['df_text_processed'].head()

1788       apa yang seharusnya kita kerjakan saat liburan
1789                      apa makanan khas daerah kalian 
1790                      apa makanan khas daerah kalian 
1791                      apa makanan khas daerah kalian 
1792    menurut kamu yang akan juara club mana ditahun...
Name: df_text_processed, dtype: object

### Stopword

In [5]:
pip install sastrawi

Note: you may need to restart the kernel to use updated packages.


Sastrawi merupakan stopword untuk bahasa indonesia

### Stemming

In [6]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [7]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import gensim
from gensim.utils import simple_preprocess

stop_words = stopwords

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = df.df_text_processed.values.tolist()
data_words = list(sent_to_words(data))

# remove stop words
data_words = remove_stopwords(data_words)

print(data_words[:1][0][:30])

['kerjakan', 'liburan']


In [9]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1)]


In [19]:
from pprint import pprint

# number of topics
num_topics = 20

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.010*"mudah" + 0.010*"makanan" + 0.009*"it" + 0.009*"bisnis" + '
  '0.007*"daerah" + 0.006*"alasannya" + 0.006*"web" + 0.006*"tidur" + '
  '0.006*"penyebab" + 0.006*"orang"'),
 (1,
  '0.014*"indonesia" + 0.007*"sih" + 0.006*"al" + 0.006*"qur" + '
  '0.006*"memilih" + 0.006*"an" + 0.006*"bahasa" + 0.004*"menghadapi" + '
  '0.004*"favorit" + 0.004*"jurnal"'),
 (2,
  '0.023*"indonesia" + 0.010*"orang" + 0.009*"virus" + 0.009*"corona" + '
  '0.007*"buah" + 0.006*"pemerintah" + 0.006*"terbaik" + 0.006*"masyarakat" + '
  '0.005*"air" + 0.005*"perbedaan"'),
 (3,
  '0.011*"indonesia" + 0.011*"br" + 0.009*"manfaat" + 0.008*"daerah" + '
  '0.006*"perbedaan" + 0.006*"src" + 0.005*"img" + 0.005*"dampak" + '
  '0.005*"kesehatan" + 0.005*"buah"'),
 (4,
  '0.006*"sosial" + 0.006*"adakah" + 0.006*"penyakit" + 0.006*"anak" + '
  '0.005*"sebutkan" + 0.005*"kerja" + 0.005*"makanan" + 0.005*"orang" + '
  '0.005*"indonesia" + 0.004*"of"'),
 (5,
  '0.011*"negara" + 0.009*"mengatasi" + 0.007*"orang"

In [20]:
import pyLDAvis.gensim_models as gensimvism
import pyLDAvis
import pickle 


# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvism.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')

LDAvis_prepared

C:\Users\acer\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
14     0.000678  0.150851       1        1  5.795594
3      0.019974 -0.102361       2        1  5.694356
2      0.061780 -0.079742       3        1  5.559932
9      0.074002  0.036202       4        1  5.532558
6      0.040331 -0.000909       5        1  5.418044
10     0.086016  0.017842       6        1  5.334000
4      0.000511 -0.030592       7        1  5.257798
11     0.084266 -0.021535       8        1  5.214175
0      0.026131 -0.028136       9        1  5.083363
1     -0.139202  0.031230      10        1  5.072725
17    -0.089188 -0.104307      11        1  4.950756
8      0.003136  0.019677      12        1  4.944993
5      0.027230 -0.006554      13        1  4.845697
18     0.065195  0.046692      14        1  4.786058
7     -0.095397  0.001202      15        1  4.773250
15    -0.013317 -0.021213      16        1  4.761670
12    -0.051984  0.011814      17        1  4.738694
19    -0.042368  0.094091      18        1  4.270094
13    -0.056028 -0.032832      19        1  4.175551
16    -0.001765  0.018579      20        1  3.790695, topic_info=           Term        Freq       Total Category  logprob  loglift
54         anak   63.000000   63.000000  Default  30.0000  30.0000
128   mengatasi   34.000000   34.000000  Default  29.0000  29.0000
40        orang  102.000000  102.000000  Default  28.0000  28.0000
4327         br   14.000000   14.000000  Default  27.0000  27.0000
322        arti   17.000000   17.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
40        orang    1.806628  102.078642  Topic20  -5.6007  -0.7617
618      kuliah    1.441770   13.203804  Topic20  -5.8263   1.0580
85       bisnis    1.441755   12.226610  Topic20  -5.8263   1.1349
575        mata    1.441728   21.862018  Topic20  -5.8264   0.5537
661       teman    1.441728   16.783284  Topic20  -5.8264   0.8181

[1425 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
2542     13  0.685270   aakah
328       3  0.669177  abisan
1038     19  0.675396     abu
2245      1  0.672909    acak
1131     13  0.685246  action
...     ...       ...     ...
4549      3  0.672804    zoom
4549     14  0.336402    zoom
3609      4  0.687396   سياسة
3681     16  0.672136     كما
3682     16  0.672104    مصدر

[3488 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 4, 3, 10, 7, 11, 5, 12, 1, 2, 18, 9, 6, 19, 8, 16, 13, 20, 14, 17])